# Работа с реляционными БД

In [1]:
import sqlite3
import sqlalchemy

conn = sqlite3.connect("students.db")  # Как оригинально!
cursor = conn.cursor()

А дальше соединения с любой реляционной БД и (для любых типов реляционных СУБД) должны (желательно) удовлетворять [Python DB-API 2.0](https://www.python.org/dev/peps/pep-0249/).

In [2]:
with open("11.drop_ddl.sql", 'r', encoding='utf-8') as f: drop_ddl = f.read()
with open("11.create_ddl.sql", 'r', encoding='utf-8') as f: create_ddl = f.read()
with open("11.insert_dml.sql", 'r', encoding='utf-8') as f: insert_dml = f.read()

In [3]:
if True:
    for st in drop_ddl.split(';'):
        cursor.execute(st)
    conn.commit()

In [4]:
for st in create_ddl.split(';'):
    cursor.execute(st)
conn.commit()

In [5]:
for st in insert_dml.split(';'):
    cursor.execute(st)
conn.commit()

In [6]:
from IPython.display import HTML, display
import tabulate  # pip install tabulate

cursor.execute(
    '''select id, name from programs'''
)
display(HTML(tabulate.tabulate(cursor.fetchall(), tablefmt='html')))

cursor.execute(
    '''select programs.name, students.name, surname, patronymic
    from programs join students
    on programs.id = students.program_id''')
display(HTML(tabulate.tabulate(cursor.fetchall(), tablefmt='html')))

1,математика и информационные технологии
2,теоретическая физика


математика и информационные технологии,Антон,Битов,
теоретическая физика,Виолетта,Аргонова,


In [7]:
conn.close()

Теперь попробуем немного [ORM](https://en.wikipedia.org/wiki/Object-relational_mapping) — [SQLAlchemy](https://ru.wikibooks.org/wiki/SQLAlchemy)

In [8]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base

DeclBase = declarative_base()

In [9]:
engine = sqlalchemy.create_engine('sqlite:///students.db', echo=False)  # echo=True для логгинга
Session = sessionmaker(bind=engine)
session = Session()

In [10]:
class Program(DeclBase):
    __tablename__ = 'programs'
    id = Column(Integer, primary_key=True)
    name = Column(String)

    students = relationship("Student", backref="program")
    programs_courses = relationship("Program_course", backref="course2")

    def __init__(self, name):
        self.name = name
        
class Course(DeclBase):
    __tablename__ = 'courses'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    programs_courses = relationship("Program_course", backref="course")
    marks = relationship("Mark", backref="course2")
    
    def __init__(self, name):
        self.name = name
        
class Program_course(DeclBase):
    __tablename__ = 'programs_courses'
    semester_number = Column(Integer, primary_key=True)
    course_id = Column(Integer, ForeignKey('courses.id'), primary_key=True)
    program_id = Column(Integer, ForeignKey('programs.id'), primary_key=True)
    
    def __init__(self, semester_number, course_id, program_id):
        self.semester_number = semester_number
        self.course_id = course_id
        self.program_id = program_id
        
class Mark(DeclBase):
    __tablename__ = 'marks'
    student_id = Column(Integer, ForeignKey('students.id'), primary_key=True)
    course_id = Column(Integer, ForeignKey('courses.id'), primary_key=True)
    mark = Column(Integer)
    
    students = relationship("Student", backref="mark")
    
    def __init__(self, student_id, course_id, mark):
        self.student_id = student_id
        self.course_id = course_id
        self.mark = mark

class Student(DeclBase):
    __tablename__ = 'students'
    id = Column(Integer, primary_key=True)
    card = Column(String)
    surname = Column(String)
    name = Column(String)
    patronymic = Column(String)
    
    program_id = Column(Integer, ForeignKey('programs.id'))
    
    marks = relationship("Mark", backref="student")
    
    def __init__(self, card, surname, name, patronymic, program):
        self.card = card
        self.surname = surname
        self.name = name
        self.patronymic = patronymic
        self.program = program

In [11]:
se = Program("Программная инженерия")

st1 = Student("002002", "Иванов", "Пётр", "Сидорович", se)
st2 = Student("002003", "Петрова", "Исидора", "Ивановна", se)

session.add_all([se, st1, st2])
session.commit()

In [12]:
import sys

print("Программы и студенты")
for p in session.query(Program):
    print("Программа: ", p.name)
    for s in p.students:
        print("- ", s.name)

Программы и студенты
Программа:  математика и информационные технологии
-  Антон
Программа:  теоретическая физика
-  Виолетта
Программа:  Программная инженерия
-  Пётр
-  Исидора


In [13]:
print("Программы и предметы")
for p in session.query(Program):
    print(p.name)
    for c in p.programs_courses:
        print(session.query(Course).filter_by(id=c.course_id)[0].name)

Программы и предметы
математика и информационные технологии
Алгебра
Матан
Алгоритмы
теоретическая физика
Алгебра
Матан
Физика
Программная инженерия


In [14]:
print("Студенты и оценки")
for s in session.query(Student):
    print(s.name)
    for m in s.marks:
        print(m.mark)

Студенты и оценки
Антон
5
Виолетта
4
Пётр
Исидора


In [15]:
for c in session.query(Course):
    print(c.name)
    for m in c.marks:
        print(m.mark, session.query(Student).filter_by(id=m.student_id)[0].name)

Алгебра
5 Антон
Матан
Алгоритмы
4 Виолетта
Физика


In [16]:
for s in session.query(Student).filter_by(surname="Аргонова"):
    s.program = se
session.commit()

print("Программы и студенты")
for p in session.query(Program):
    print("Программа: ", p.name)
    for s in p.students:
        print("- ", s.name)

Программы и студенты
Программа:  математика и информационные технологии
-  Антон
Программа:  теоретическая физика
Программа:  Программная инженерия
-  Виолетта
-  Пётр
-  Исидора
